In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

from summer.utils import ref_times_to_dti

from autumn.tools.plots.utils import REF_DATE
from autumn.tools.runs import ManagedRun
from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt
from autumn.settings.region import Region

## Specify the runs of interest

In [ ]:
scenario_names = ["baseline", "roadmap scenario"]
plot_left_date = datetime.date(2021, 8, 2)

In [ ]:
mr = ManagedRun("covid_19/victoria/1639527196/7088603")
pbi = mr.powerbi.get_db()
results = pbi.get_uncertainty()
targets = pbi.get_targets()

In [ ]:
full_run = mr.full_run.get_derived_outputs()

In [ ]:
plot_left_date = datetime.date(2021, 9, 1)
plot_right_date = datetime.date(2022, 6, 30)
fig, axes = plt.subplots(3, 2, figsize=(12, 14), sharex="all")

outputs = (
    "notifications", "infection_deaths", "hospital_admissions", "icu_admissions", "hospital_occupancy", "icu_occupancy"
)

for i_ax, axis in enumerate(axes.reshape(-1)):
    output = outputs[i_ax]

    scenario_chain = (full_run["scenario"] == 0) & (full_run["chain"] == 0)
    for i_run in full_run[scenario_chain]["run"].unique():
        selection = full_run[(full_run["run"] == i_run) & scenario_chain]
        axis.plot(ref_times_to_dti(REF_DATE, selection["times"]), selection[output])
    if output in targets:
        axis.scatter(targets.index, targets[output], color="k", s=10, alpha=0.5, zorder=10)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date, right=plot_right_date)
    axis.set_title(output.replace("_", " "))

fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 14), sharex="all")
colours = ((0.2, 0.2, 0.8), (0.8, 0.2, 0.2))
scenarios = [0, 1]
for i_ax, axis in enumerate(axes.reshape(-1)):
    output = outputs[i_ax]
    indices = results.index

    for scenario in scenarios:
        results_df = results[(output, scenario)]
        colour = colours[scenario]
        axis.fill_between(indices, results_df[0.01], results_df[0.99], color=colour, alpha=0.1)
        axis.fill_between(indices, results_df[0.025], results_df[0.975], color=colour, alpha=0.5)
        axis.fill_between(indices, results_df[0.25], results_df[0.75], color=colour, alpha=0.7)
        axis.plot(indices, results_df[0.500], color="k", linestyle="--")

    if output in targets:
        axis.scatter(targets.index, targets[output], color="k", alpha=0.5, s=10)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date)
    axis.set_title(output)
# fig.suptitle(f"{output} under {scenario_names[scenario]} conditions")
fig.tight_layout()

In [ ]:
mcmc_params = mr.calibration.get_mcmc_params()

param_dict = {
    "contact_rate": "contact rate",
    "infectious_seed": "infectious seed",
    "mobility.microdistancing.face_coverings_adjuster.parameters.effect": "face coverings effectiveness",
    "sojourn.compartment_periods_calculated.exposed.total_period": "incubation period",
    "vaccination.vacc_full_effect_duration": "duration of full immunity from vaccination",
    "clinical_stratification.props.hospital.multiplier": "inflation of hospitalisation rates",
}

fig, axes = plt.subplots(3, 2, figsize=(12, 18))
for i_ax, axis in enumerate(axes.reshape(-1)):
    axis.hist(mcmc_params.iloc[:, i_ax])
    axis.set_title(param_dict[mcmc_params.columns[i_ax]])
